In [1]:
import json
import requests
from statistics import mean 
import operator

## Data Cleaning and Preparation

In [2]:
def return_url_params(var_url,player_id,start_date,year,oponent_team_id,venue_id,matchtype=None):
    year_wise_url = var_url+"api/year_wise_playermatchstats/"
    year_wise_params = {'player_id':player_id,'start_date':start_date,'year':year,'matchtype':matchtype,'lr':matchtype,'hr':matchtype}
    opponent_wise_url = var_url + 'api/opponent_wise_playermatchstats/'
    opponent_wise_params = {'player_id':player_id,'team_id':oponent_team_id,'start_date':start_date,'matchtype':matchtype,'lr':matchtype,'hr':matchtype}
    venue_wise_url = var_url + "api/venue_wise_playermatchstats/"
    venue_wise_params = {'player_id':player_id,'venue_id':venue_id,'start_date':start_date,'matchtype':matchtype,'lr':matchtype,'hr':matchtype}
    overall_url = var_url+'api/playermatchstats/'
    overall_params = {'player_id':player_id,'start_date':start_date,'lr':matchtype,'hr':matchtype}
    url_parms = {
        'v_u':venue_wise_url,
        'v_p':venue_wise_params,
        'o_u':opponent_wise_url,
        'o_p':opponent_wise_params,
        'y_u':year_wise_url,
        'y_p':year_wise_params,
        'ov_u':overall_url,
        'ov_p':overall_params
    }
    return url_parms

In [3]:
local_var = 'http://localhost:8000/'
global_var = 'https://win11.neolen.com/'

In [4]:
def get_stats(url,params):
    response = requests.request("GET", url, params = params)
    json_data = json.loads(response.text)
    return json_data

def return_int(d):
    try:
        d = int(d)
        return d
    except:
        return 0

In [5]:
def return_stats(json_data):     
    match_count = 0
    bowl_avg_list = []
    order_list = []
    bat_inning_count = 0
    bowl_inning_count = 0
    bat_avg_list = []
    list_50 = []
    list_100 = []
    list_5w = []
    list_3w = []
    for m in json_data:
        scorecard = m.get('scorecard',[])
        match_count += 1
        bowling_avg = 0 
        batting_avg = 0
        for s in scorecard:
            bat = s.get('match_batting_inning')
            bowl = s.get('match_bowling_inning')
            
            bat = bat[0] if len(bat) != 0 else None
                
            if bat:
                
                order = bat.get('order')
                run = return_int(bat.get('runs'))
                f_w = bat.get('fall_of_wicket_over')
                how_out = bat.get('how_out')

                #calculate batting average
                if f_w != '' and how_out != '':
                    bat_avg_list.append(float(run))
                elif how_out != "not out" and how_out != 'retired hurt':
                    bat_avg_list.append(float(run))
                else:
                    bat_avg_list.append(0.0)
                #end

                #calculate 50
                list_50.append(1 if run >= 50 and run < 100 else 0)
                #end

                #calculate 100
                list_100.append(1 if run>=100 else 0)
                #end
                
                bat_inning_count+=1
                order_list.append(order)

            bowl = bowl[0] if len(bowl) != 0 else None

            if bowl:
                
                run_conceded = return_int(bowl.get('run_conceded',0))
                wickets = return_int(bowl.get('wickets',0))
                
                #calculate bowling average
                if wickets != 0:
                    bowling_avg = run_conceded/wickets
                    bowl_avg_list.append(bowling_avg)
                else:
                    bowl_avg_list.append(0.0)
                #end
                
                #5 and 3 w
                list_3w.append(1 if wickets >= 3 and wickets < 5 else 0)
                list_5w.append(1 if wickets >= 5 else 0)
                #end
                
                bowl_inning_count+=1

#     print(match_count,bat_avg_list,bowl_avg_list,list_100,list_50,list_3w,list_5w,order_list,sep = '\n')
    bat_data = {
        'total_match':match_count,
        'innings':bat_inning_count,
        'avg':mean(bat_avg_list) if len(bat_avg_list) != 0 else 0,
        '50': sum(list_50),
        'order':mean(order_list) if len(order_list) != 0 else 0,
        '100':sum(list_100),
    }
    bowl_data = {
        'total_match':match_count,
        'innings': bowl_inning_count,
        'avg': mean(bowl_avg_list) if len(bowl_avg_list) != 0 else 0,
        '5w': sum(list_5w),
        '3w': sum(list_3w),
    }
    return bat_data , bowl_data

In [6]:
def batting_score(data,MAX_BATTING_AVG=250,MAX_50_100=30.0): #20*49+5*96
    B = data['avg']/MAX_BATTING_AVG
    C = (20*data['100'] + 5*data['50'])/data['innings'] if data['innings'] != 0 else 0
    D = C/MAX_50_100
    E = 0.4*D + 0.6*B
    return E
def bowling_score(data,MAX_BOWL_AVG=150,MAX_5_3=10.0):
    H = data['avg']/MAX_BOWL_AVG
    F = (20*data['5w'] + 5*data['3w'])/data['innings'] if data['innings'] != 0 else 0
    G = F/MAX_5_3
    I = 0.4*G + 0.6*H
    return I

In [7]:
def total_batting_score(data,player_stats):
    overall = player_stats['overall'][0]
    A = overall['innings']/overall['total_match'] if overall['total_match'] != 0 else 0
    E = data['overall']
    R = data['venue']
    P = data['year']
    Q = data['oponent']
    L = data['last_5']
    ans = A*((0.25*E) + (0.15*P) + (0.1*(Q+R))+(0.5*L))
    return round(ans,4)
def total_bowling_score(data,player_stats):
    overall = player_stats['overall'][1]
    V = overall['innings']/overall['total_match'] if overall['total_match'] != 0 else 0
    I = data['overall']
    S = data['year']
    T = data['oponent']
    U = data['venue']
    L1 = data['last_5']
    ans = V*((0.25*I) +(0.15*S) + (0.1 *
    (T+U) )+(0.5*L1))
    return round(ans,4)

In [8]:
def returnplayerusing_score(bat_score,bowl_score,allrounder_score):
    
    all_len = len(allrounder_score)
    bat_player_len = 0
    bowl_player_len = 0
    playing_11 = []
    if all_len == 3:
        bat_player_len = 4
        bowl_player_len = 4
    elif all_len < 3:
        bat_player_len = 5
        bowl_player_len = 11 - all_len - 5
    else:
        all_len = 3
        bat_player_len = 4
        bowl_player_len = 4
    for x in allrounder_score:
        playing_11.append(x[1])
        all_len -= 1
        if all_len == 0:
            break
    for y in bat_score:
        playing_11.append(y[1])
        bat_player_len-=1
        if bat_player_len == 0:
            break
    for y in bowl_score:
        playing_11.append(y[1])
        bowl_player_len-=1
        if bowl_player_len == 0:
            break
    
    return playing_11

In [9]:
def accuracy(player1 , player2):
    ans  = 0
    for p in player1:
        if p in player2:
            ans+=1
    return (ans/11)*100

In [10]:
def get_series_player(url,params):
    series_data = get_stats(local_var + url,params)
    series_data = series_data['results']
    team1_players = set()
    team2_players = set()
    for m in series_data:
        url_match_player = 'api/permatchplayer/'
        players_team1_match = get_stats(local_var+url_match_player,{'match_id':m['id'],'team_id':m['team1_id']})['results'][0]['players']
        players_team2_match = get_stats(local_var+url_match_player,{'match_id':m['id'],'team_id':m['team2_id']})['results'][0]['players']
        for p in players_team1_match:
            team1_players.add(p)
        for p in players_team2_match:
            team2_players.add(p)
    return team1_players,team2_players,series_data

In [11]:
def displayOfseriesresult(accuracy_of_matches):
    for x in accuracy_of_matches:
        print(x[2],x[0]," --> ",x[1])

In [12]:
# url_match = 'api/peryearmatch/'
# matches_19 = get_stats(local_var+url_match,{'year':2019})['results']

In [13]:
def player_stats_score(m,teamplayers,matchtype):
    bat_score_player_id = {}
    bow_score_player_id = {}
    bat_score = {}
    bowl_score = {}
    for p in teamplayers:

        batting_score_id = {}
        bowling_score_id = {}
        player_all_stats = return_url_params(local_var,p,m['start_date'],2019,m['team2_id'],m['venue_id'],matchtype=matchtype)

        #overallstats and last 5 match stats
        overall_stats = return_stats(get_stats(player_all_stats['ov_u'],player_all_stats['ov_p']).get('result',{}))
        if len(overall_stats) < 5:
            h = len(overall_stats)
        elif len(overall_stats) == 0:
            h = 0
        else:
            h = 5
        if h != 0:
            last_5_stats = overall_stats[:h]
        #end

        player_stats = {
            'year': return_stats(get_stats(player_all_stats['y_u'],player_all_stats['y_p'])['result']),
            'oponent': return_stats(get_stats(player_all_stats['o_u'],player_all_stats['o_p'])['result']),
            'venue': return_stats(get_stats(player_all_stats['v_u'],player_all_stats['v_p'])['result']),
            'overall': overall_stats,
            'last_5': last_5_stats
        }
        batting_score_id = {
            'overall' :batting_score(player_stats['overall'][0]),
             'venue'  :batting_score(player_stats['venue'][0]),
            'oponent':batting_score(player_stats['oponent'][0]),
             'year'   :batting_score(player_stats['year'][0]),
            'last_5': batting_score(player_stats['last_5'][0])
        }
        bowling_score_id = {
            'overall' :bowling_score(player_stats['overall'][1]),
             'venue'  :bowling_score(player_stats['venue'][1]),
            'oponent':bowling_score(player_stats['oponent'][1]),
             'year'   :bowling_score(player_stats['year'][1]),
            'last_5': bowling_score(player_stats['last_5'][1])
        }
        bat_s = total_batting_score(batting_score_id,player_stats)
        bowl_s = total_bowling_score(bowling_score_id,player_stats)
        bat_score_player_id[p] = bat_s
        bat_score[bat_s] = p
        bow_score_player_id[p] = bowl_s
        bowl_score[bowl_s] = p

    return bat_score,bowl_score,bat_score_player_id,bow_score_player_id

In [14]:
def playing11_model(series_id,matchtype):
    
    accuracy_of_matches = []
    team1players , team2players , matches_perSeries = get_series_player('api/matchperyearOrseries/',{'series_id':series_id,'lr':matchtype,'hr':matchtype})
    
    for m in matches_perSeries:
        url_match_player = 'api/permatchplayer/'
        players_team1_match = get_stats(local_var+url_match_player,{'match_id':m['id'],'team_id':m['team1_id']})['results'][0]['players']
        players_team2_match = get_stats(local_var+url_match_player,{'match_id':m['id'],'team_id':m['team2_id']})['results'][0]['players']

        for t in [(team1players,players_team1_match,m['team1_id']),(team2players,players_team2_match,m['team2_id'])]:
            print('%'*100)
            print(m['description'],t[2],sep = '\n')
            print()
            bat_score , bowl_score , bat_score_player_id , bow_score_player_id = player_stats_score(m,t[0],matchtype)  

            all_rounder_score_player = {}
            #calculate the all rounder using threeshold
            for p in t[0]:
                if bat_score_player_id.get(p) != None and bow_score_player_id.get(p) != None:
                    if bat_score_player_id[p] <= 0.13 and bat_score_player_id[p] >= 0.04 and bow_score_player_id[p] <= 0.13 and bow_score_player_id[p] >= 0.04:
                        avg_score = (bat_score_player_id[p] + bow_score_player_id[p])/2
                        all_rounder_score_player[round(avg_score,4)] = p
                        del bat_score[bat_score_player_id[p]]
                        del bowl_score[bow_score_player_id[p]]
                        del bat_score_player_id[p]
                        del bow_score_player_id[p]
            #end

            #sort the score
            bat_score = sorted(bat_score.items(), key=operator.itemgetter(0),reverse=True)
            bowl_score = sorted(bowl_score.items(), key=operator.itemgetter(0),reverse=True)
            all_rounder_score_player = sorted(all_rounder_score_player.items(), key=operator.itemgetter(0),reverse=True)
            #end

            playing_11 = returnplayerusing_score(bat_score,bowl_score,all_rounder_score_player)

            print('batting score of player - >' , bat_score)
            print('bowling score of player - >' , bowl_score)
            print('allrounder score of player - >' , all_rounder_score_player)
            print()
            print("actual playing 11 --> ",t[1])
            print('predictive playing 11 of team {} - >'.format(t[2]),playing_11)
            acc = accuracy(playing_11,t[1])
            accuracy_of_matches.append((t[2],acc,m['description']))
            print('accuracy of predictive playing 11 of team {} -> '.format(t[2]),acc)
            print('\n\n\n')
            print('%'*100)
    return accuracy_of_matches

In [ ]:
accuracyOfMatch = playing11_model(31,50.0)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
3rd ODI (D/N), India tour of New Zealand at Mount Maunganui, Feb 11 2020
4

batting score of player - > [(0.1701, 37), (0.1684, 36), (0.167, 771), (0.1412, 38), (0.1161, 774), (0.108, 34), (0.0728, 35), (0.0409, 41), (0.0128, 42), (0.0121, 757), (0.0099, 1987), (0.001, 770)]
bowling score of player - > [(0.1942, 42), (0.1428, 757), (0.1318, 770), (0.1315, 41), (0.0971, 1987), (0.0375, 36), (0.0155, 774), (0.0027, 771), (0.002, 35), (0.0015, 37), (0.0, 38)]
allrounder score of player - > [(0.0857, 764), (0.0466, 40)]

actual playing 11 -->  [34, 770, 36, 37, 38, 771, 40, 41, 42, 1987, 764]
predictive playing 11 of team 4 - > [764, 40, 37, 36, 771, 38, 774, 42, 757, 770, 41]
accuracy of predictive playing 11 of team 4 ->  81.81818181818183




%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [ ]:
# team1_Acc = []
# team2_ACC = []
# team = []
# for x in accuracy_of_matches:
#     if x[0] not in team:
#         team.append(x[0])
# total_acc_team1 = 0
# total_acc_team2 = 0
# for x in accuracy_of_matches:
#     if team[0] == x[0]:
#         total_acc_team1+=x[1]
#     if team[1] == x[0]:
#         total_acc_team2+=x[1]
# total_acc_team1 = total_acc_team1/3
# total_acc_team2 = total_acc_team2/3
# print('total acc of team1 is --> ',total_acc_team1)
# print('total acc of team2 is --> ',total_acc_team2)

In [ ]:
!git add .

In [ ]:
!git commit -m " model changed little bit concept"

In [ ]:
# !git push origin master